![](https://nitheshsinghsanjay.github.io/images/mobtiny_fig.PNG)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

What we’re going to do is download a MobileNet model, and then use it for inference just on a few random images to see how well it classifies these images according to ImageNet classes.

We first make a call to `tf.keras.applications.mobilenet.MobileNet()` to obtain a copy of a single pretrained MobileNet with weights that were saved from being trained on ImageNet images. We’re assigning this model to the variable mobile

In [1]:
mobile = tf.keras.applications.mobilenet.MobileNet()

Next, we have a function called `prepare_image()` that accepts an image file, and processes the image to get it in a format that the model expects. We’ll be passing each of our images to this function before we use MobileNet to predict on it, so let’s see what exactly this function is doing.

In [1]:
def prepare_image(file):
    img_path = '/kaggle/input/horseimage/'
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)


Within this function, we first define the relative path to the images. I have all the sample images we’ll be using stored in kaggle/input.

We then call the Keras function `image.load_img()` which accepts the image file and a target_size for the image, which we’re setting to (224,224) (which is the default size for MobileNet). `load_img()` returns an instance of a PIL image.

We then convert the PIL image into an array with the Keras img_to_array() function, and then we expand the dimensions of that array by using numpy’s `expand_dims()`.

Lastly, we’re calling `preprocess_input()` from `tf.keras.applications.mobilenet`, which preprocesses the given image data to be in the same format as the images that MobileNet was originally trained on. Specifically, it’s scaling the pixel values in the image between -1 and 1, and this function will return the preprocessed image data as a numpy array.

This result is what we’re returning within this overall `prepare_image()` function.

# Predicting with MobileNet

Let’s now get some predictions from MobileNet and see how it performs. We’ll be using some random sample images

In [1]:
from IPython.display import Image
Image(filename='/kaggle/input/horseimage/horse.jpg', width=300,height=200) 

We’re going to process this image by passing it to our `prepare_image()` function and assign the result to this preprocessed_image variable. We’re then having MobileNet predict on this image by calling `mobile.predict()` and passing it our preprocessed_image.

In [1]:
preprocessed_image = prepare_image('horse.jpg')
predictions = mobile.predict(preprocessed_image)

Then, we’re using an ImageNet utility function provided by Keras called `decode_predictions()`. It returns the top five ImageNet class predictions with the ImageNet class ID, the class label, and the probability. With this, we’ll be able to see the five ImageNet classes with the highest prediction probabilities from our model on this given image. Recall that there are 1000 total ImageNet classes.

In [1]:
results = imagenet_utils.decode_predictions(predictions)

In [1]:
results

Checking out the results, we have Saluki with 25%, then Great Dane at 19%, whippet at 13%, and a couple of other types.

# Building A Fine-Tuned MobileNet Model

Now that we’ve seen what MobileNet is all about in above cells, let’s now talk about how we can fine-tune the model and and use transfer learning to train it on another dataset.
If you’re not already familiar with the concept of fine-tuning, that’s alright because we have [another notebook](https://www.kaggle.com/bavalpreet26/vgg-16-keras-nb3) on fine-tuning using the VGG16 model with Keras

Similar to what we previously implemented with VGG16, we’re going to be fine-tuning MobileNet on images of cats and dogs. The implementation will be pretty similar, but you’ll notice there will be a few differences.

Many different breeds of cats and dogs were included in the ImageNet data set for which MobileNet was originally trained on, so the original model has already learned a lot about cats and dogs in general. Because of this, it won’t take much tuning to get the model to perform well on this specific, more narrow classification task.

In a later cells, however, we’ll be fine-tuning MobileNet on a completely new data set made up of classes that the model hasn’t already learned about in it’s original training, so stay tuned for that.

# Preparing The Data

We now define the path variables for where the training and test set reside on directory.

In [1]:
train_path = '/kaggle/input/cat-and-dog/training_set/training_set'
test_path = '../input/cat-and-dog/test_set/test_set'

Then, we create directory iterators for each dataset using Keras’ `ImageDataGenerator.flow_from_directory()` function, which yeilds batches of image data from the directory that we pass in with our first parameter.

In [1]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cats', 'dogs'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cats', 'dogs'], batch_size=10, shuffle=False)

Notice the preprocessing_function parameter we’re supplying to ImageDataGenerator. We’re setting this equal to `keras.applications.mobilenet.preprocess_input()`. This is going to do the necessary MobileNet preprocessing on the images obtained from `flow_from_directory()`.

Recall, we talked about this exact function in the above cells and its role in regards to preprocessing images for MobileNet.

To flow_from directory(), we’re passing in the path to the data set, the target_size for the images, and the batch_size we’re choosing to use for training.

For the test_batches variable, we’re also supplying one additional parameter, `shuffle=False`, so that we can later access the corresponding non-shuffled test lables to plot a confusion matrix.

The data portion is now done. Next, let’s move on to modifying the model.

# Model Modification

In [1]:
mobile = tf.keras.applications.mobilenet.MobileNet()

Next, we’re going to <font color='green'>grab the output from the sixth to last layer</font> of the model and store it in this variable <font color ='red'> x </font>.

In [1]:
x = mobile.layers[-6].output

We’ll be using this to build a new model. This new model will consist of the original MobileNet up to the sixth to last layer. We’re not including the last five layers of the original MobileNet.

In [1]:
mobile.summary()

By looking at the summary of the original model, we can see that by not including the last five layers, we’ll be including everything up to and including the last` global_average_pooling` layer.

Note that the amount of layers that you choose to cut off when you’re fine-tuning a model will vary for each scenario, but I’ve found through experimentation that just removing the last `5 layers` here works out well for this particular task. So with this setup, we’ll be keeping the vast majority of the original MobileNet architecutre, which has 88 layers total.

Now, we append an output layer that we’re calling output, which will just be a `Dense layer` with 2 output nodes, for cat and dog, and we’ll use the `softmax` activation function.



In [1]:
output = Dense(units=2, activation='softmax')(x)

Now, we construct the new fine-tuned model, which we’re calling `model`.

In [1]:
model = Model(inputs=mobile.input, outputs=output)

Note, you can see by the Model constructor used to create our model, that this is a model that is being created with the Keras Functional API, not the Sequential API that we’ve worked with in previous notebook. That’s why this format that we’re using to create the model may look a little different than what you’re used to.

*To build the new model, we create an instance of the Model class and specify the inputs to the model to be equal to the input of the original MobileNet, and then we define the outputs of the model to be equal to the output variable we created directly above.*

This creates a new model, which is identical to the original MobileNet up to the original model’s sixth to last layer. We don’t have the last five original MobileNet layers included, but instead we have a new layer, the output layer we created with two output nodes.

You can compare the summary of the new model here with the summary of the original MobileNet to verifiy these differences using by calling summary() on both the old and new models

In [1]:
model.summary()

Now, we need to choose how many layers we actually want to be trained when we train on cats and dogs.

Here, we are freezing the weights of all the layers except for the last five layers in our new model, meaning that only the last five layers of the model will be trained.

In [1]:
for layer in model.layers[:-5]:
    layer.trainable = False

By training only the last five layers, all the weights in the remaining earlier layers will not be updated during training and instead will be saved with the ImageNet weights from the original MobileNet.

Note that the number of layers that you choose to retrain is, again, one of those things that <font color = 'orange'>varies by situtation</font>. Since the original MobileNet model has already generally learned about cats and dogs, we’re not really needing to retrain many layers.

Now, our new model is now built, tuned, and ready to be trained on cats and dogs

# Training The Model

In [1]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

On this model, we’re first calling compile and specifying the Adam optimizer with a `learning rate of .0001`. We’re setting the loss to `categorical_crossentropy`, and our metrics just include `accuracy`.

After our model is compiled, we’re now going to train the model by calling fit().

In [1]:
model.fit(x=train_batches,
          steps_per_epoch=len(train_batches),
          epochs=10,
          verbose=2
)

With the minor tuning we did to the model, it is performing very well on this new task.

# Using The Model For Inference

Next, we’re going to use our model to predict on images from our test set that it hasn’t already seen during training or validation.

Before we run the predictions, we’re going to get and format the labels for the test set, and we need these just in order to plot the confusion matrix we’ll see in a few moments. We don’t actually need the labels to get predictions from the model.

First, we define `test_labels` to be equal to `test_batches.classes`.

In [1]:
test_labels = test_batches.classes

Calling classes on test_batches is going to give us the class names (i.e. the labels), for each sample in the test set. This is the reason we specified `shuffle=False` for test_batches

Since we’ll be using these static labels that are returned from test_batches.classes, we can’t shuffle our test data each time we use it for predicting because then the labels won’t map correctly to the data.

By calling `class_indices` on test_batches, we can see the mapping from the underlying class names, cat and dog, to the 0s and 1s. The output of `test_batches.class_indices` looks like this.

* {'cat': 0, 'dog': 1}

So, we can see that a cat label corresponds to `0`, and a dog label corresponds to `1`.

We’ve now got our labels taken care of. Let’s now get some predictions.

In [1]:
predictions = model.predict(x=test_batches, verbose=0)

We define this predictions variable to be equal to `model.predict()`, and we’re passing in our test_batches.

Lastly, we set the verbosity equal to 0, which is just not going to print out any output.

# Visualize Predictions In Confusion Matrix

We’ll be using the scikit-learn library to do that. If you’re not generally familiar with confusion matrices, or you want to learn more about working with them, check out the earlier [notebook](https://www.kaggle.com/bavalpreet26/cnn-tutorial-keras-nb2) where we go into more detail on using scikit-learn’s confusion matrix.

First, we have a function called `plot_confusion_matrix()`, and this is what we’ll be calling in a few moments to do the plotting.

In [1]:
def plot_confusion_matrix(cm, classes,
        normalize=False,
        title='Confusion matrix',
        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

Note that this code was pulled directly off of [scikit-learn’s](https://scikit-learn.org/stable/) website, so we’re not going to go over the details here.

Next, we create this confusion_matrix object called `cm`, and we set it equal to scikit-learn’s confusion_matrix that we imported at the start of our code in the last episode.

In [1]:
cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))

To our confusion matrix, we pass the labels of our test set as well as the prediction results stored in `predictions`. Calling `argmax` on the predictions is going to return the indices that contain the maximum values from the list of predictions. So, because we only have two classes, it will return a 0 or 1 for each prediction in the predictions list.

Next, we define the labels for our `confusion matrix`, and these need to be in the order that the class indices are in. Recall the class indices from `test_batches` we checked out above.

In [1]:
cm_plot_labels = ['cat','dog']

Since cat was first and then dog in the class indices dictionary, that’s how we specify the order for the confusion matrix labels.

Then, we call the `plot_confusion_matrix()` function that we referenced above, and we pass in our confusion matrix `cm`, the labels `cm_plot_labels`, and we give it the title of 'Confusion Matrix'

In [1]:
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

We can conclude our fine-tuned model overall did a really great job on the task of classifying images as cats or dogs.